In [22]:
# Load Params
push!(LOAD_PATH, pwd())

using BenchmarkTools
using DarkWingDuck

### Benchmark Creation and Query Statistics - 500,000 points 
- Insert Test: Insert 500,000 random points on ((0, 1), (0, 1))
- Object Size: 
- Box Search: Query bounding box w. SW corner on ((0, 1), (0, 1)) with sideLength (0, 0.5)
- Radial Search: Query circle w. center on ((0, 1), (0, 1)) and radius of 5,000m (~0.045 in cartesian plane)

In [20]:
"""Generate uniform random points along a surface"""
function genRandomCoords(x0::Float64, x1::Float64, y0::Float64, y1::Float64, n::Int)::Array{DarkWingDuck.Coord}
    X = ((ones(n) * x0) + (rand(n) * (x1 - x0))) 
    Y = ((ones(n) * y0) + (rand(n) * (y1 - y0)))
    return [DarkWingDuck.Coord(x, y) for (x,y) in zip(X,Y)]
end

generateRandUniform

In [14]:
const QT_TEST_SIZE, QT_TEST_RM_SIZE = 4, 500_000, 100_000

## Inserts - Tree Build
ps = genRandomCoords(0., 1., 0., 1., QT_TEST_SIZE);

#TODO: constructor should take anyFloat
sampleQTree = DarkWingDuck.qtBox(DarkWingDuck.Coord(Float32(0.), Float32(0.)), Float32(1.))

@time for p in ps 
    DarkWingDuck.insertIntoQuadTree!(sampleQTree, p) 
end

  1.107086 seconds (12.58 M allocations: 410.189 MiB, 24.98% gc time)


In [15]:
"Total Obj Size: $(Base.summarysize(sampleQTree) / 1000000 ) MB"

"Total Obj Size: 26.157864 MB"

In [17]:
@benchmark DarkWingDuck.queryRange(sampleQTree, DarkWingDuck.Box(DarkWingDuck.Coord(rand() , rand()), rand()/2))

BenchmarkTools.Trial: 
  memory estimate:  8.09 KiB
  allocs estimate:  148
  --------------
  minimum time:     9.703 μs (0.00% GC)
  median time:      4.017 ms (0.00% GC)
  mean time:        6.928 ms (7.44% GC)
  maximum time:     42.451 ms (11.15% GC)
  --------------
  samples:          720
  evals/sample:     1

In [16]:
#Test in kilometers, 5km ~0.045 on a flat plane
@benchmark DarkWingDuck.radialSearch(sampleQTree, DarkWingDuck.Coord(rand(), rand()), 5.) 

BenchmarkTools.Trial: 
  memory estimate:  138.21 MiB
  allocs estimate:  1180816
  --------------
  minimum time:     126.227 ms (4.61% GC)
  median time:      131.744 ms (6.61% GC)
  mean time:        135.798 ms (9.15% GC)
  maximum time:     233.347 ms (44.77% GC)
  --------------
  samples:          37
  evals/sample:     1

In [19]:
# Oyy, Benchmarking function!() is pretty expensive, but passing Modify == false is ~3-4x quicker
removeCoords = ps[map(x -> Int64(floor(x + 1)), rand(QT_TEST_RM_SIZE) * QT_TEST_RM_SIZE)]

@benchmark for rmc in removeCoords 
    DarkWingDuck.removePointRebuild!(sampleQTree, rmc, true) 
end

BenchmarkTools.Trial: 
  memory estimate:  885.88 MiB
  allocs estimate:  28973008
  --------------
  minimum time:     2.404 s (3.40% GC)
  median time:      2.415 s (3.47% GC)
  mean time:        2.426 s (3.43% GC)
  maximum time:     2.459 s (3.42% GC)
  --------------
  samples:          3
  evals/sample:     1